# Advanced MetPy Calculations

In [ ]:
from datetime import datetime, timedelta

import metpy.calc as mpcalc
from metpy.plots.declarative import *
from metpy.units import units
import xarray as xr

In [ ]:
# Set the date/time of the model run
date = datetime(2020, 2, 28, 0)

# Remote access to the dataset from the UCAR site
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                     f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55))

## Advection Calculation

Advection = Wind Vector * Temperature Gradient

Advection Calculation in MetPy (https://unidata.github.io/MetPy/v1.0/api/generated/metpy.calc.advection.html#metpy.calc.advection)
                   

In [ ]:
level = 850 * units.hPa
plot_time = date + timedelta(hours=0)

tmpk = ds.Temperature_isobaric.metpy.sel(vertical=level, time=plot_time)
uwnd = ds['u-component_of_wind_isobaric'].metpy.sel(vertical=level, time=plot_time)
vwnd = ds['v-component_of_wind_isobaric'].metpy.sel(vertical=level, time=plot_time)

lats = ds.lat.values
lons = ds.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

adv = mpcalc.advection(tmpk.values * units.K,
                       [uwnd.values * units('m/s'), vwnd.values * units('m/s')],
                       [dx, dy])

# Add it to dataset (ds)
# First set needed elements based on what is being calculated
dim_names = tmpk.dims
variable_values = adv.m * 1e4
attribute_dictionary = {'grid_mapping': tmpk.grid_mapping,
                        'units': adv.units}
# Add it to dataset
ds = ds.assign(temperature_advection=(dim_names, variable_values, attribute_dictionary))

In [ ]:
# Set attributes for plotting contours
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = 'temperature_advection'
cfill.level = None # Since already chose level
cfill.time = None # Since already chose time
cfill.contours = list(range(-15, 16, 1))
cfill.colormap = 'coolwarm'
cfill.colorbar = 'horizontal'

cntr = ContourPlot()
cntr.data = ds
cntr.field = 'Temperature_isobaric'
cntr.level = level
cntr.time = plot_time
cntr.contours = list(range(-40, 41, 5))
cntr.linecolor = 'red'
cntr.linestyle = 'dashed'
cntr.clabels = True
cntr.plot_units = 'degC'

cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = level
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 30))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

barbs = BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_isobaric', 'v-component_of_wind_isobaric']
barbs.level = level
barbs.skip = (3, 3)
barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'850-hPa Temperature Advection at {plot_time} by KHG'
panel.plots = [cfill, cntr, cntr2, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

## Relative Vorticity Calculation

Relative_Vorticity = (Change in V Wind)/(Change in X-distance) - (Change in U Wind)/(Change in Y-distance)

MetPy Relative Vorticity Calculation: https://unidata.github.io/MetPy/v1.0/api/generated/metpy.calc.vorticity.html#metpy.calc.vorticity

In [ ]:
level = 500 * units.hPa
plot_time = date + timedelta(hours=0)

uwnd = ds['u-component_of_wind_isobaric'].metpy.sel(vertical=level, time=plot_time)
vwnd = ds['v-component_of_wind_isobaric'].metpy.sel(vertical=level, time=plot_time)

lats = ds.lat.values
lons = ds.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

relative_vorticity = mpcalc.vorticity(uwnd.values * units('m/s'), vwnd.values * units('m/s'), dx, dy)

# Add it to dataset (ds)
# First set needed elements based on what is being calculated
dim_names = uwnd.dims
variable_values = relative_vorticity.m * 1e5
attribute_dictionary = {'grid_mapping': uwnd.grid_mapping,
                        'units': relative_vorticity.units}
# Add it to dataset
ds = ds.assign(relative_vorticity=(dim_names, variable_values, attribute_dictionary))

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = 'relative_vorticity'
cfill.level = None
cfill.time = None
cfill.contours = list(range(-40, 41, 2))
cfill.colormap = 'PuOr_r'
cfill.colorbar = 'horizontal'

cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 500 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 60))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

barbs = BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_isobaric', 'v-component_of_wind_isobaric']
barbs.level = level
barbs.skip = (3, 3)
barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'{cntr2.level.m}-hPa Heights and Wind Speed at {plot_time} by KHG'
panel.plots = [cfill, cntr2, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()